
  

*   This Python notebook provides a comprehensive solution for extracting audio from YouTube playlists, converting the audio into text through transcription, and organizing the output by including the title of each video followed by its transcribed content.
*   It is designed to handle playlists of any size, with an option to limit the number of videos processed.
*   The process involves downloading the audio tracks of selected YouTube videos, splitting large audio files into smaller, manageable chunks based on periods of silence, and then using speech recognition technology to convert these audio chunks into written text.
*   The code efficiently manages multiple files by processing them in parallel, leveraging the capabilities of modern multi-core processors to speed up the transcription process.
*     This approach is user-friendly, requiring only the URL of the YouTube playlist and the desired number of videos to process, making it accessible for individuals without technical expertise.

## Installing packages and Libraries

In [3]:
!pip install speechrecognition pydub pytube
!pip install reportlab

In [17]:
import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence
import tempfile
import os
from pytube import YouTube, Playlist
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from concurrent.futures import ThreadPoolExecutor, as_completed

### To Transcribe YouTube Video

It usually takes 5-6 mins to transcribe a YouTube Video of 10 mins.

In [8]:
# Function to download audio from YouTube
def download_audio_from_youtube(url):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).first()
    output_path = tempfile.gettempdir()
    audio_stream.download(output_path=output_path)
    return os.path.join(output_path, audio_stream.default_filename)

# Function to transcribe a single audio chunk
def transcribe_audio(path):
    r = sr.Recognizer()
    with sr.AudioFile(path) as source:
        audio_listened = r.record(source)
        try:
            text = r.recognize_google(audio_listened)
            return text
        except sr.UnknownValueError as e:
            return "Error: " + str(e)
        except sr.RequestError as e:
            return "API Error: " + str(e)

# Parallel transcription function
def transcribe_chunk(chunk_filename):
    text = transcribe_audio(chunk_filename)
    if not text.startswith("Error"):
        text = f"{text.capitalize()}. "
    return text

# Main function to handle large audio files and transcribe them using parallel processing
def get_large_audio_transcription_on_silence(path):
    sound = AudioSegment.from_file(path)
    chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=sound.dBFS-14, keep_silence=500)

    folder_name = tempfile.mkdtemp()
    chunk_filenames = []
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        chunk_filenames.append(chunk_filename)

    whole_text = ""
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        future_text = {executor.submit(transcribe_chunk, filename): filename for filename in chunk_filenames}
        for future in future_text:
            whole_text += future.result()

    # Cleanup
    for file in os.listdir(folder_name):
        os.remove(os.path.join(folder_name, file))
    os.rmdir(folder_name)

    return whole_text



In [9]:
YoutubeURL = input("Enter YouTube URL Link: ")
audio_path = download_audio_from_youtube(YoutubeURL)
transcription = get_large_audio_transcription_on_silence(audio_path)
if transcription:
    print("Successful")
else:
    print("Please try again")

Enter YouTube URL Link: https://www.youtube.com/watch?v=yRM3sc57q0c&list=PLXFMmlk03Dt7Q0xr1PIAriY5623cKiH7V&index=1
Successful


In [10]:
transcription

"Hi my name is tim roughgarden i'm a professor here at stanford university and i'd like to welcome you to this first course on the design and analysis of. Now i imagine many of you are already clear on your reasons for taking this. But let me begin by justifying this courses existence. I'm giving you some reasons why you should be highly motivated. To learn about algorithm. So what is an algorithm anyway. Basically it's a set of well-defined rules recipe in effect. For solving some computational problem. Maybe you have a bunch of numbers and you want to rearrange them so that they're in sorted order. Maybe you have a road map and an origin and a destination. And you want a computer shortest path from that origin. True that destination. Maybe you face a number of different tasks that need to be completed by certain deadlines. And you want to know in what order you should accomplish the tasks. So that you complete them all by their respective. So why study algorithm. Well first of all un

Downloading the transcribed content in PDF format

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

# Define the PDF file name
pdf_file_name = '/content/transcription.pdf'

# Create a PDF document template
doc = SimpleDocTemplate(pdf_file_name, pagesize=letter)
story = []  # This will hold the content for the PDF

# Set up a basic style for the paragraphs
styles = getSampleStyleSheet()
style = styles["Normal"]

# Choose font and font size
style.fontName = 'Times-Roman'
style.fontSize = 14

# Split the transcription into paragraphs and add each to the story
for paragraph in transcription.split('\n'):
    story.append(Paragraph(paragraph, style))

# Build the PDF
doc.build(story)

### To transcribe a YouTube Playlist

In [18]:
# Function to download audio from a limited number of YouTube playlist videos
def download_audio_from_playlist(playlist_url, max_videos=None):
    playlist = Playlist(playlist_url)
    audio_paths_with_titles = []
    video_urls = playlist.video_urls[:max_videos] if max_videos else playlist.video_urls

    for video_url in video_urls:
        yt = YouTube(video_url)
        audio_stream = yt.streams.filter(only_audio=True).first()
        output_path = tempfile.gettempdir()
        audio_path = audio_stream.download(output_path=output_path)
        title = yt.title  # Get the video title
        audio_paths_with_titles.append((audio_path, title))

    return audio_paths_with_titles

# Function to transcribe a single audio chunk
def transcribe_audio(path):
    r = sr.Recognizer()
    with sr.AudioFile(path) as source:
        audio_listened = r.record(source)
        try:
            text = r.recognize_google(audio_listened)
            return text
        except sr.UnknownValueError as e:
            return "Error: " + str(e)
        except sr.RequestError as e:
            return "API Error: " + str(e)

# Parallel transcription function
def transcribe_chunk(chunk_filename):
    text = transcribe_audio(chunk_filename)
    if not text.startswith("Error"):
        text = f"{text.capitalize()}. "
    return text

# Main function to handle large audio files and transcribe them using parallel processing
def get_large_audio_transcription_on_silence(path):
    sound = AudioSegment.from_file(path)
    chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=sound.dBFS-14, keep_silence=500)

    folder_name = tempfile.mkdtemp()
    chunk_filenames = []
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        chunk_filenames.append(chunk_filename)

    whole_text = ""
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        future_text = {executor.submit(transcribe_chunk, filename): filename for filename in chunk_filenames}
        for future in as_completed(future_text):
            whole_text += future.result()

    # Cleanup
    for file in os.listdir(folder_name):
        os.remove(os.path.join(folder_name, file))
    os.rmdir(folder_name)

    return whole_text

# Function to transcribe audio files from a YouTube playlist
def transcribe_audio_files(audio_paths_with_titles):
    transcriptions = []
    for audio_path, title in audio_paths_with_titles:
        transcription = get_large_audio_transcription_on_silence(audio_path)
        transcriptions.append((title, transcription))

    return transcriptions

# Main function to process a YouTube playlist
def process_youtube_playlist(playlist_url, max_videos=None):
    audio_paths_with_titles = download_audio_from_playlist(playlist_url, max_videos)
    transcriptions = transcribe_audio_files(audio_paths_with_titles)

    for title, transcription in transcriptions:
        print(f"Title: {title}\nTranscription: {transcription}\n")

In [20]:
#Enter your Playlist URL here
playlist_url = input("Enter your playlist url : ")
max_videos = 2
print(process_youtube_playlist(playlist_url, max_videos))

Enter your playlist url : https://www.youtube.com/watch?v=yRM3sc57q0c&list=PLXFMmlk03Dt7Q0xr1PIAriY5623cKiH7V&index=1
Title: 1   1   Why Study Algorithms   4 min
Transcription: Now i imagine many of you are already clear on your reasons for taking this. Hi my name is tim roughgarden i'm a professor here at stanford university and i'd like to welcome you to this first course on the design and analysis of. But let me begin by justifying this courses existence. To learn about algorithm. I'm giving you some reasons why you should be highly motivated. So what is an algorithm anyway. For solving some computational problem. Basically it's a set of well-defined rules recipe in effect. Maybe you have a bunch of numbers and you want to rearrange them so that they're in sorted order. Maybe you have a road map and an origin and a destination. True that destination. And you want a computer shortest path from that origin. Maybe you face a number of different tasks that need to be completed by certai